In [4]:
import socket
import json


host = "localhost"
port = 8081

def send_and_recieve(json_data):

    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect((host,port))
    login_info = json_data
    login_info = json.dumps(login_info)
    try:
        client.send(bytes(login_info, encoding="ascii"))
        response = client.recv(4096)
        response = response.decode("ascii")
        response = json.loads(response)
    except:
        print("Server returned nothing or request was out of time")
        return ""
    return response

# Testing "login" and action validation

In [5]:
assert send_and_recieve({"malformed": "request"}) == {'state': 'error', 'info': 'badRequest'} # request without fields "name", "action", "actionSpecs"
assert send_and_recieve({'name': "userNotLoggable", "action": "listRooms", "actionSpecs": {}}) == {'state': 'error', 'info': 'userNotAllowedError'} # unauthorized user
assert send_and_recieve({'name': "riccardo", "action": "unknownAction", "actionSpecs": {}}) == {"state": "error", "info": "unknownAction"} # authorized user but unknown action

# testing room create / destroy

In [6]:
assert send_and_recieve({'name': "riccardo", "action": "createRoom", "actionSpecs": {"room": "room1"}}) == {"state": "ok", "info": "roomCreated"}  # create room ok
assert send_and_recieve({'name': "alessandro", "action": "destroyRoom", "actionSpecs": {"room": "room1"}}) == {"state": "error", "info": "notAllowed"} # other user cannot destroy room
assert send_and_recieve({'name': "riccardo", "action": "destroyRoom", "actionSpecs": {"room": "randomRoom"}}) == {"state": "error", "info": "notAllowed"} # cannot destory non-existent room
assert send_and_recieve({'name': "riccardo", "action": "destroyRoom", "actionSpecs": {"room": "room1"}}) == {"state": "ok", "info": "roomDestroyed"} # ok room destroyed
assert send_and_recieve({'name': "riccardo", "action": "destroyRoom", "actionSpecs": {"room": "room1"}}) == {"state": "error", "info": "notAllowed"} # double destroy, impossible


# testing room list

In [7]:
assert send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}}) == {"state": "ok", "rooms": "[]"}
send_and_recieve({'name': "riccardo", "action": "createRoom", "actionSpecs": {"room": "room1"}})
assert "room1" in send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}})["rooms"]
send_and_recieve({'name': "riccardo", "action": "createRoom", "actionSpecs": {"room": "room2"}})
assert "room1" in send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}})["rooms"]
assert "room2" in send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}})["rooms"]
send_and_recieve({'name': "riccardo", "action": "destroyRoom", "actionSpecs": {"room": "room1"}})
assert "room1" not in send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}})["rooms"]
assert "room2" in send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}})["rooms"]
send_and_recieve({'name': "riccardo", "action": "destroyRoom", "actionSpecs": {"room": "room2"}})
assert send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}}) == {"state": "ok", "rooms": "[]"}

# testing room join and chat

In [8]:
send_and_recieve({'name': "riccardo", "action": "createRoom", "actionSpecs": {"room": "room1"}})
assert send_and_recieve({'name': "riccardo", "action": "joinRoom", "actionSpecs": {"room": "room1"}}) == {'info': 'joinedRoom', 'state': 'ok'}
assert send_and_recieve({'name': "alessandro", "action": "joinRoom", "actionSpecs": {"room": "room1"}}) == {'info': 'joinedRoom', 'state': 'ok'}
assert send_and_recieve({'name': "riccardo", "action": "sendMessage", "actionSpecs": {"room": "room1", "message": "ciao1"}}) == {'info': 'messageSent', 'state': 'ok'}
assert send_and_recieve({'name': "riccardo", "action": "sendMessage", "actionSpecs": {"room": "room1", "message": "ciao2"}}) == {'info': 'messageSent', 'state': 'ok'}
assert send_and_recieve({'name': "riccardo", "action": "sendMessage", "actionSpecs": {"room": "room1", "message": "ciao3"}}) == {'info': 'messageSent', 'state': 'ok'}
assert send_and_recieve({'name': "alessandro", "action": "receiveMessage", "actionSpecs": {"room": "room1"}}) == {'messages': ['ciao1', 'ciao2', 'ciao3'], 'state': 'ok'}
assert send_and_recieve({'name': "alessandro", "action": "leaveRoom", "actionSpecs": {"room": "room1"}}) == {'info': "leftRoom", 'state': 'ok'}
assert send_and_recieve({'name': "alessandro", "action": "receiveMessage", "actionSpecs": {"room": "room1"}}) == {'info': 'notInRoom', 'state': 'error'}

# test private messages

In [9]:
# testing private messages
send_and_recieve({'name': "riccardo", "action": "sendPrivateMessage", "actionSpecs": {"receiver": "alessandro", "message": "hello"}})
assert send_and_recieve({'name': "alessandro", "action": "receivePrivateMessage", "actionSpecs": {"sender": "riccardo"}}) == {'messages': ['hello'], 'state': 'ok'}
send_and_recieve({'name': "riccardo", "action": "sendPrivateMessage", "actionSpecs": {"receiver": "alessandro", "message": "how are you?"}})
assert send_and_recieve({'name': "alessandro", "action": "receivePrivateMessage", "actionSpecs": {"sender": "riccardo"}}) == {'messages': ['hello', 'how are you?'], 'state': 'ok'}
send_and_recieve({'name': "alessandro", "action": "sendPrivateMessage", "actionSpecs": {"receiver": "riccardo", "message": "I'm fine, thanks!"}})
assert send_and_recieve({'name': "riccardo", "action": "receivePrivateMessage", "actionSpecs": {"sender": "alessandro"}}) == {'messages': ["I'm fine, thanks!"], 'state': 'ok'}

# test private rooms

In [10]:
send_and_recieve({'name': "riccardo", "action": "createRoom", "actionSpecs": {"room": "room2", "access": "private"}}) # creating private room
assert send_and_recieve({'name': "riccardo", "action": "joinRoom", "actionSpecs": {"room": "room2"}}) == {'info': 'joinedRoom', 'state': 'ok'} # join private room
assert send_and_recieve({'name': "alessandro", "action": "joinRoom", "actionSpecs": {"room": "room2"}}) == {'info': 'roomNotFound', 'state': 'error'} # cannot join private room without invite
assert send_and_recieve({'name': "riccardo", "action": "inviteInRoom", "actionSpecs": {"room": "room2", "user": "alessandro"}}) == {'info': 'inviteeJoinedRoom', 'state': 'ok'} # join via invite
assert send_and_recieve({'name': "alessandro", "action": "sendMessage", "actionSpecs": {"room": "room2", "message": "hello"}}) == {'info': 'messageSent', 'state': 'ok'} # send message
assert send_and_recieve({'name': "riccardo", "action": "receiveMessage", "actionSpecs": {"room": "room2"}}) == {'messages': ['hello'], 'state': 'ok'} # receive message